In [7]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer, text_to_word_sequence, one_hot, hashing_trick
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


In [8]:
raw_numpy = pd.read_csv("numpy.csv")
raw_pandas = pd.read_csv("pandas.csv")
raw_csharp = pd.read_csv("csharp.csv")
raw_matlab = pd.read_csv("matlab.csv")
raw_matplotlib = pd.read_csv("matplotlib.csv")
raw_mongodb = pd.read_csv("mongodb.csv")
raw_tensorflow = pd.read_csv("tensorflow.csv")
raw_typescript = pd.read_csv("typescript.csv")

In [9]:
print(raw_numpy.shape)
print(raw_pandas.shape)
print(raw_csharp.shape)
print(raw_matlab.shape)
print(raw_matplotlib.shape)
print(raw_mongodb.shape)
print(raw_tensorflow.shape)
print(raw_typescript.shape)

(45005, 1)
(47904, 1)
(45664, 1)
(45921, 1)
(43507, 1)
(44070, 1)
(41255, 1)
(47392, 1)


In [10]:
raw_numpy["Tag"]="numpy"
raw_pandas["Tag"]="pandas"
raw_csharp["Tag"]="csharp"
raw_matlab["Tag"]="matlab"
raw_matplotlib["Tag"]="matplotlib"
raw_mongodb["Tag"]="mongodb"
raw_tensorflow["Tag"]="tensorflow"
raw_typescript["Tag"]="typescript"

In [11]:
print(raw_numpy.shape)
print(raw_pandas.shape)
print(raw_csharp.shape)
print(raw_matlab.shape)
print(raw_matplotlib.shape)
print(raw_mongodb.shape)
print(raw_tensorflow.shape)
print(raw_typescript.shape)

(45005, 2)
(47904, 2)
(45664, 2)
(45921, 2)
(43507, 2)
(44070, 2)
(41255, 2)
(47392, 2)


In [12]:
df = pd.concat([raw_numpy, raw_pandas, raw_csharp, raw_matlab, raw_matplotlib, raw_mongodb, raw_tensorflow, raw_typescript], axis=0)
df = df.reset_index(drop=True)
#df = df[pd.notnull(df['Tag'])]
print(df.shape)
df.head()

(360718, 2)


,Questions,Tag
0,numpy savez() of a list of 3D arrays with diff...,numpy
1,'DataFrame' object has no attribute 'Close' in...,numpy
2,How to Look up multiple values for one column ...,numpy
3,What are fastest data types to use as an alter...,numpy
4,rgb2hed not giving the same answer when applie...,numpy


In [13]:
print(df['Questions'].apply(lambda x: len(x.split(' '))).sum())

3636886


In [14]:
def print_plot(index):
    example = df[df.index == index][['Questions', 'Tag']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)
print_plot(300000)

Sum rows of 2D array with elements of 1D array
Tag: numpy
Running automatic annotation in cvat with tensorflow results in status code 400 "No labels found for tf annotation"
Tag: tensorflow


In [15]:
tokens = df["Questions"].apply(lambda x: word_tokenize(x))
tokens

0         [numpy, savez, (, ), of, a, list, of, 3D, arra...
1         ['DataFrame, ', object, has, no, attribute, 'C...
2         [How, to, Look, up, multiple, values, for, one...
3         [What, are, fastest, data, types, to, use, as,...
4         [rgb2hed, not, giving, the, same, answer, when...
                                ...                        
360713    [How, to, access, angular, Component, from, ot...
360714    [Testing-library, custom, query, Typescript, e...
360715    [React, Native, navigation, (, 4.x, ), props, ...
360716    [TypeORM, duplicate, Key, value, for, ManyToMa...
360717    [Problem, :, sending, object, in, post, reques...
Name: Questions, Length: 360718, dtype: object

In [16]:
words = tokens.apply(lambda x: [w for w in x if w.isalpha()])
words

0         [numpy, savez, of, a, list, of, arrays, with, ...
1         [object, has, no, attribute, in, pandas, How, ...
2         [How, to, Look, up, multiple, values, for, one...
3         [What, are, fastest, data, types, to, use, as,...
4         [not, giving, the, same, answer, when, applied...
                                ...                        
360713    [How, to, access, angular, Component, from, ot...
360714                   [custom, query, Typescript, error]
360715    [React, Native, navigation, props, become, nul...
360716    [TypeORM, duplicate, Key, value, for, ManyToMa...
360717    [Problem, sending, object, in, post, request, ...
Name: Questions, Length: 360718, dtype: object

In [17]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [18]:
filtered_words = words.apply(lambda x: [word for word in x if word not in stop_words])
filtered_words

0         [numpy, savez, list, arrays, different, shapes...
1            [object, attribute, pandas, How, fix, problem]
2         [How, Look, multiple, values, one, column, pyt...
3         [What, fastest, data, types, use, alternative,...
4             [giving, answer, applied, full, part, matrix]
                                ...                        
360713    [How, access, angular, Component, Component, d...
360714                   [custom, query, Typescript, error]
360715    [React, Native, navigation, props, become, nul...
360716    [TypeORM, duplicate, Key, value, ManyToMany, r...
360717    [Problem, sending, object, post, request, angu...
Name: Questions, Length: 360718, dtype: object

In [19]:
stemmer = PorterStemmer()
stemmed_words = filtered_words.apply(lambda x: [stemmer.stem(word) for word in x])
stemmed_words

0         [numpi, savez, list, array, differ, shape, fai...
1              [object, attribut, panda, how, fix, problem]
2         [how, look, multipl, valu, one, column, python...
3         [what, fastest, data, type, use, altern, vecto...
4                 [give, answer, appli, full, part, matrix]
                                ...                        
360713       [how, access, angular, compon, compon, duplic]
360714                   [custom, queri, typescript, error]
360715     [react, nativ, navig, prop, becom, null, screen]
360716      [typeorm, duplic, key, valu, manytomani, relat]
360717    [problem, send, object, post, request, angular...
Name: Questions, Length: 360718, dtype: object

In [23]:
clean_sentence = stemmed_words.apply(lambda x: " ".join(x))
clean_sentence

0         numpi savez list array differ shape fail simpl...
1                     object attribut panda how fix problem
2            how look multipl valu one column python duplic
3           what fastest data type use altern vector python
4                        give answer appli full part matrix
                                ...                        
360713              how access angular compon compon duplic
360714                        custom queri typescript error
360715             react nativ navig prop becom null screen
360716             typeorm duplic key valu manytomani relat
360717    problem send object post request angular sprin...
Name: Questions, Length: 360718, dtype: object

In [21]:
df.head()

,Questions,Tag
0,numpy savez() of a list of 3D arrays with diff...,numpy
1,'DataFrame' object has no attribute 'Close' in...,numpy
2,How to Look up multiple values for one column ...,numpy
3,What are fastest data types to use as an alter...,numpy
4,rgb2hed not giving the same answer when applie...,numpy


In [24]:
df["Questions"] = clean_sentence
df.head()

,Questions,Tag
0,numpi savez list array differ shape fail simpl...,numpy
1,object attribut panda how fix problem,numpy
2,how look multipl valu one column python duplic,numpy
3,what fastest data type use altern vector python,numpy
4,give answer appli full part matrix,numpy


In [26]:
# Tokenize the text data and convert it to sequences
max_words = 30000  # Maximum number of words to consider
max_len = 100  # Maximum length of input sequences
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.Questions)
sequences = tokenizer.texts_to_sequences(df.Questions)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 34641 unique tokens.


In [27]:
data = pad_sequences(sequences, maxlen=max_len)

In [28]:
# Convert labels to categorical format
num_classes = len(set(df.Tag))
unq_tags = df.Tag.unique()
word_to_label = {word: i for i, word in enumerate(unq_tags)}
labels = [word_to_label[word] for word in df.Tag]
labels = np.asarray(labels)
labels = to_categorical(labels, num_classes=num_classes)
print(labels.shape)
labels

(360718, 8)


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [29]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [30]:
# Build the CNN model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='sigmoid'))

In [31]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
9018/9018 [==============================] - 382s 42ms/step - loss: 0.6047 - accuracy: 0.7961 - val_loss: 0.5448 - val_accuracy: 0.8152
Epoch 2/10
9018/9018 [==============================] - 389s 43ms/step - loss: 0.4950 - accuracy: 0.8306 - val_loss: 0.5462 - val_accuracy: 0.8134
Epoch 3/10
9018/9018 [==============================] - 373s 41ms/step - loss: 0.4410 - accuracy: 0.8491 - val_loss: 0.5734 - val_accuracy: 0.8083
Epoch 4/10
9018/9018 [==============================] - 378s 42ms/step - loss: 0.3946 - accuracy: 0.8635 - val_loss: 0.5891 - val_accuracy: 0.8071
Epoch 5/10
9018/9018 [==============================] - 385s 43ms/step - loss: 0.3548 - accuracy: 0.8773 - val_loss: 0.6355 - val_accuracy: 0.7998
Epoch 6/10
9018/9018 [==============================] - 384s 43ms/step - loss: 0.3223 - accuracy: 0.8883 - val_loss: 0.6789 - val_accuracy: 0.7912
Epoch 7/10
9018/9018 [==============================] - 376s 42ms/step - loss: 0.2960 - accuracy: 0.8969 - val_loss: 0

In [33]:
# Evaluate the model
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.859277069568634
Test accuracy: 0.7807301878929138
